In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
import copy
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils.visualize_util import model_to_dot, plot
from keras import backend as K
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 4007)


In [3]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-27'

In [4]:
RawDataPath=WORKSPACE_PATH+'/raw/'

In [5]:
StudyDataPath=WORKSPACE_PATH+'/study/'

In [6]:
import processing

センサデータを読み込む

In [7]:
dic1=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor1")
dic2=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor2")
dic3=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor3")
dic4=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor4")
dic5=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor5")
dic6=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor6")
dic7=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor7")

windows関数を作成する。
要求仕様
* 入力データは2次元配列　raw 時間、colm　データの種類（加速度X,加速度Y,・・・）
* 出力データは4次元配列　numberOfWindow * windowWidth * kindOfData * 
# how to use
1. Set Data(1darray)
case 1  
    seed.append(AccX)  
    seed.append(AccY)  
    seed.append(AccZ)  
2. Making window
w = build(seed)  
w -> [ [AccX],[AccY],[AccZ] ]  
window = compile(w)  
window -> 
          [ [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]], ....]

In [8]:
class Window:
    """Input dictionary
       Ouput Windows
    """         
    def __init__(self):                  
        self.seed ={}
    
    def SetData(self, RegistDataName, RegistData):
        if RegistDataName in self.seed:
            print RegistDataName+' has been registed before'
            return -1
        
        self.seed[RegistDataName] = RegistData
        print RegistDataName+' is registed now'
    
    def _Build(self):
        keys = self.seed.keys()
        array = self.seed[keys[0]]
        
        for k in keys[1:]:
            array = np.vstack( (array,self.seed[k]))
        print 'Build Complete'
        return array.T
    
    def Compile(self,windowWidth,overlap):
        source = self._Build()
        print source.shape
       
        sourceRows = source.ndim
        sourceColum = source.size
        offset = len(source)%windowWidth
        if offset % 2 == 1:
            offset=offset/2 +1
        else:
            offset=offset/2
        # first array
        start = offset
        goal = start+windowWidth
        window = source[start:goal].T
        
        for i in range(1,len(source)/windowWidth):
            start = start+ int(windowWidth*overlap)
            goal = start+windowWidth
            adding = source[start:goal].T
            window = np.dstack((window,adding))
        return window.T

In [9]:
w = Window()

In [10]:
w.SetData('dic1-AccX',dic1['AccX'])
w.SetData('dic1-AccY',dic1['AccY'])
w.SetData('dic1-AccZ',dic1['AccZ'])

dic1-AccX is registed now
dic1-AccY is registed now
dic1-AccZ is registed now


In [11]:
w.SetData('dic2-AccX',dic2['AccX'])
w.SetData('dic2-AccY',dic2['AccY'])
w.SetData('dic2-AccZ',dic2['AccZ'])

dic2-AccX is registed now
dic2-AccY is registed now
dic2-AccZ is registed now


In [12]:
w.SetData('dic3-AccX',dic3['AccX'])
w.SetData('dic3-AccY',dic3['AccY'])
w.SetData('dic3-AccZ',dic3['AccZ'])

dic3-AccX is registed now
dic3-AccY is registed now
dic3-AccZ is registed now


In [13]:
w.SetData('dic4-AccX',dic4['AccX'])
w.SetData('dic4-AccY',dic4['AccY'])
w.SetData('dic4-AccZ',dic4['AccZ'])

dic4-AccX is registed now
dic4-AccY is registed now
dic4-AccZ is registed now


In [14]:
w.SetData('dic5-AccX',dic5['AccX'])
w.SetData('dic5-AccY',dic5['AccY'])
w.SetData('dic5-AccZ',dic5['AccZ'])

dic5-AccX is registed now
dic5-AccY is registed now
dic5-AccZ is registed now


In [15]:
w.SetData('dic6-AccX',dic6['AccX'])
w.SetData('dic6-AccY',dic6['AccY'])
w.SetData('dic6-AccZ',dic6['AccZ'])

dic6-AccX is registed now
dic6-AccY is registed now
dic6-AccZ is registed now


In [16]:
w.SetData('dic7-AccX',dic7['AccX'])
w.SetData('dic7-AccY',dic7['AccY'])
w.SetData('dic7-AccZ',dic7['AccZ'])

dic7-AccX is registed now
dic7-AccY is registed now
dic7-AccZ is registed now


In [17]:
window = w.Compile(windowWidth=16,overlap=0.5)

Build Complete
(254742, 21)


出来上がったwindowの形をみれば、Mnistを模倣したほうがいいかもしれない  
というのは、windowの集合を見るとMnistに近いものになっていた  
Mnistの訓練データは[sample, 28,28]  
一方、出来上がったwindowは[sample,windowWidth,data]  
また、cifar10は[sample,32,32,3]  


kerasとMnistに関して、入力層をどのようにやっているのかわからないので実際にデータをダウンロードしてどのような構造でダウンロードして、入力層に与えるときに何をしているかをはっきりさせる

In [20]:
def Convert3dto2d(window):
    output = window.reshape((len(window),np.prod(window.shape[1:])))
    return output

In [21]:
window_train = window.reshape((len(window),np.prod(window.shape[1:])))
#window_train = window_train[:12000]
window_test = window.reshape((len(window),np.prod(window.shape[1:])))
#window_test = window_test[12001:]

すべてのセンサデータの加速度データ、角速度データを食わせる

In [1]:
from keras.layers import Input, Dense
from keras.models import Model

windoW = window
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='All_encoded'
SaveFileNameDecord='All_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

# this is the size of our encoded representations
encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

shapeNum=window_train.shape[1]

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoderALL = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoderALL = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoderALL.predict(window_test)
decoded_imgs = decoderALL.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 4007)


NameError: name 'window' is not defined

あるセンサの一次元のみでやってみる

In [2]:
from keras.layers import Input, Dense
from keras.models import Model

w1 = Window()
w1.SetData('AccX', dic1['AccX'])
window1 = w1.Compile(windowWidth=16,overlap=0.5)

windoW = window1

col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor1_AccX_encoded'
SaveFileNameDecord='sensor1_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder1 = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder1 = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder1 = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder1.compile(optimizer='adadelta', loss='mse')

autoencoder1.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder1.predict(window_test)
decoded_imgs = decoder1.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

NameError: name 'Window' is not defined

2つ目のセンサーで同じようにやってみる

In [24]:
w2 = Window()
w2.SetData('AccX', dic2['AccX'])
window2 = w2.Compile(windowWidth=16,overlap=0.5)

windoW = window2
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor2_AccX_encoded'
SaveFileNameDecord='sensor2_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0167 - val_loss: 0.0018
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 0.0014 - val_loss: 0.0010
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 9.0481e-04 - val_loss: 7.2637e-04
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 6.6992e-04 - val_loss: 5.5499e-04
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 5.2983e-04 - val_loss: 4.4674e-04
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 4.5479e-04 - val_loss: 3.9030e-04
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 4.0173e-04 - val_loss: 8.3055e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 3.5444e-04 - val_loss: 3.6848e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 3.2153e-04 - val_loss: 2.70

３つ目のデータ

In [25]:
w3 = Window()
w3.SetData('AccX', dic3['AccX'])
window3 = w3.Compile(windowWidth=16,overlap=0.5)

windoW = window3
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor3_AccX_encoded'
SaveFileNameDecord='sensor3_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0517 - val_loss: 0.0053
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 0.0038 - val_loss: 0.0028
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 0.0023 - val_loss: 0.0019
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 0.0015 - val_loss: 0.0012
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 0.0010 - val_loss: 8.3615e-04
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 7.6357e-04 - val_loss: 6.2531e-04
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 6.3255e-04 - val_loss: 5.4667e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 5.5633e-04 - val_loss: 5.1464e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 5.0698e-04 - val_loss: 0.0039
Epoch 10/50
15921

4つ目のセンサデータ

In [26]:
w4 = Window()
w4.SetData('AccX', dic4['AccX'])
window4 = w4.Compile(windowWidth=16,overlap=0.5)

windoW = window4
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor4_AccX_encoded'
SaveFileNameDecord='sensor4_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0409 - val_loss: 0.0050
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 0.0040 - val_loss: 0.0032
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 0.0027 - val_loss: 0.0023
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 0.0020 - val_loss: 0.0016
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 0.0014 - val_loss: 0.0013
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 0.0011 - val_loss: 0.0011
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 9.4109e-04 - val_loss: 8.6122e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 8.2479e-04 - val_loss: 7.7045e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 7.4043e-04 - val_loss: 7.9515e-04
Epoch 10/50
15921/15921 [

５つ目のセンサデータ

In [28]:
w5 = Window()
w5.SetData('AccX', dic5['AccX'])
window5 = w5.Compile(windowWidth=16,overlap=0.5)

windoW = window5
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor5_AccX_encoded'
SaveFileNameDecord='sensor5_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0136 - val_loss: 8.0928e-04
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 6.5904e-04 - val_loss: 5.4253e-04
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 4.4473e-04 - val_loss: 3.7194e-04
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 3.1303e-04 - val_loss: 2.5549e-04
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 2.2779e-04 - val_loss: 2.1869e-04
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 1.7412e-04 - val_loss: 1.4005e-04
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 1.4042e-04 - val_loss: 1.2342e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 1.2280e-04 - val_loss: 1.2752e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 1.1339e-04 - va

６つ目のセンサでやってみる

In [29]:
w6 = Window()
w6.SetData('AccX', dic6['AccX'])
window6 = w6.Compile(windowWidth=16,overlap=0.5)

windoW = window6
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor6_AccX_encoded'
SaveFileNameDecord='sensor6_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0227 - val_loss: 0.0018
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 0.0015 - val_loss: 0.0012
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 0.0010 - val_loss: 0.0010
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 8.7926e-04 - val_loss: 8.1223e-04
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 7.8701e-04 - val_loss: 7.4287e-04
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 7.0696e-04 - val_loss: 6.6835e-04
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 6.5104e-04 - val_loss: 6.5485e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 6.0073e-04 - val_loss: 5.6537e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 5.6287e-04 - val_loss: 5.2923e-04
E

７つ目のセンサーでやってみる

In [30]:
w7 = Window()
w7.SetData('AccX', dic6['AccX'])
window7 = w7.Compile(windowWidth=16,overlap=0.5)

windoW = window7
col = windoW.shape[1]
row = windoW.shape[2]
shapeNum=col*row

SaveFileNameEncord='sensor7_AccX_encoded'
SaveFileNameDecord='sensor7_AccX_decoded'

window_train=Convert3dto2d(windoW)
window_test=Convert3dto2d(windoW)

encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameEncord,
                                  encoder.predict(window_test))

processing.SaveDicDataFromFileNPZ(StudyDataPath,SaveFileNameDecord,
                                  encoder.predict(decoded_imgs))

AccX is registed now
Build Complete
(254742,)
Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 4s - loss: 0.0125 - val_loss: 0.0025
Epoch 2/50
15921/15921 [==============================] - 4s - loss: 0.0022 - val_loss: 0.0020
Epoch 3/50
15921/15921 [==============================] - 4s - loss: 0.0018 - val_loss: 0.0016
Epoch 4/50
15921/15921 [==============================] - 4s - loss: 0.0015 - val_loss: 0.0013
Epoch 5/50
15921/15921 [==============================] - 4s - loss: 0.0012 - val_loss: 0.0011
Epoch 6/50
15921/15921 [==============================] - 4s - loss: 0.0010 - val_loss: 9.6622e-04
Epoch 7/50
15921/15921 [==============================] - 4s - loss: 8.9085e-04 - val_loss: 8.4533e-04
Epoch 8/50
15921/15921 [==============================] - 4s - loss: 7.9483e-04 - val_loss: 7.3908e-04
Epoch 9/50
15921/15921 [==============================] - 4s - loss: 7.2857e-04 - val_loss: 6.8617e-04
Epoch 10/50
15921/159